In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
mouths_dict = {
    'января': 'january',
    'февраля': 'february',
    'декабря': 'december',
    'ноября': 'november',
    'октября': 'october',
    'сентября': 'september',
    'мая': 'may',
    'июня': 'june',
    'июля': 'july',
    'августа': 'august',
    'апреля': 'april',
    'марта': 'march'
}
def fix_date(date: str):
    if date:
        date = date.lower()
        for key, value in mouths_dict.items():
            date = date.replace(key, value)
        return date
    return None

In [4]:
fake = faker.Faker(locale='en')

In [5]:
sber_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'Host': 'rabota.sber.ru',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'referer': 'https://rabota.sber.ru/search?keyword='
}

In [6]:
API_URL = "https://rabota.sber.ru/v2/gateway/api/pulseservice/v1/search/get?"
VACANCY_API_URL = "https://rabota.sber.ru/v2/gateway/api/pulseservice/v1/search/get/"

In [7]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip() if x else None

In [8]:
search_params = {
    'page': 12,
    'size': 100
}

In [9]:
first_request = requests.get(API_URL, headers=sber_headers, params=search_params, verify=False)
total_pages = first_request.json()['totalPages']
print(total_pages, first_request.json()['totalCount'])

13 1292


### Получаем списки айдишников со всех страниц

In [10]:
jobs_infos = []
for page in tqdm(range(total_pages)):

    search_params = {
        'page': page,
        'size': 100
    }

    result = requests.get(API_URL, params=search_params, headers=sber_headers, verify=False)
    if result.status_code != 200:
        print(f"Skipping page {page}, response code {result.status_code}")
        continue

    for vacancy in result.json()['vacancies']:
        internal_id = vacancy['internalId']
        jobs_infos.append(internal_id)

    time.sleep(4.0) #sleep for 00 ms

100%|██████████| 13/13 [01:18<00:00,  6.06s/it]


In [11]:
len(jobs_infos)

1292

In [13]:
jobs_dicts = []
for id in tqdm(jobs_infos):
    result = requests.get(VACANCY_API_URL + str(id), headers=sber_headers, verify=False)
    if result.status_code != 200:
        print(f'error {result.status_code}')
    else:
        try:
            job = result.json()
            job_dict = {
                'title': job['title'],
                'internal_id' : id,
                'url': VACANCY_API_URL + str(id),
                'description': clear_string(job['introduction']),
                'responsibilities': clear_string(job['duties']),
                'qualifications': clear_string(job['requirements']),
                'location': job['location'],
                'company': 'Sber',
                'publish_date': parser.parse(fix_date(job['activeFrom']))
            }
            jobs_dicts.append(job_dict)
        except Exception as e:
            print(f'Error during parsing vacancy {id}: {e}')

    time.sleep(0.3) # 500 ms sleep

  4%|▎         | 47/1292 [01:52<43:38,  2.10s/it]  

error 500


 28%|██▊       | 368/1292 [16:46<35:42,  2.32s/it]  

error 500


 89%|████████▉ | 1154/1292 [50:32<04:29,  1.96s/it] 

...

 90%|████████▉ | 1157/1292 [50:39<05:20,  2.37s/it]

error 500


 91%|█████████ | 1170/1292 [51:10<05:42,  2.81s/it]

error 500


100%|██████████| 1292/1292 [56:23<00:00,  2.62s/it]


In [14]:
snapshot = pd.DataFrame(jobs_dicts)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,location,company,publish_date
441,Специалист,4093106,https://rabota.sber.ru/v2/gateway/api/pulseser...,None,расчет NPV-сценариев использования/реализации ...,высшее образование (в области экономики и фина...,"Город Челябинск, Челябинская Область",Sber,2022-08-09 08:56:51
930,Ведущий специалист судебного и исполнительного...,4080021,https://rabota.sber.ru/v2/gateway/api/pulseser...,None,Взыскание просроченной задолженности с физичес...,"Специалист со средним специальным, неоконченны...","Город Тихорецк, Тихорецкий Район, Краснодарски...",Sber,2022-07-14 07:38:22
959,Backend (Java) разработчик (команда Applicatio...,4016929,https://rabota.sber.ru/v2/gateway/api/pulseser...,Мы занимаемся Application Security во всем Сбе...,Участие в проектировании и разработке продукто...,"Hard skills \n \n Фреймворки: Spring (Boot, MV...",Город Москва,Sber,2022-07-18 11:36:43
29,Machine Learning DevOps инженер,4071760,https://rabota.sber.ru/v2/gateway/api/pulseser...,О проекте \n Мы команда экспертов в области по...,Наша команда занимается инфраструктурой для ра...,Что для нас важно: \n \n Знание и опыт работы ...,"Город Нижний Новгород, Нижегородская Область",Sber,2022-07-11 15:04:59
1251,Python разработчик,4093660,https://rabota.sber.ru/v2/gateway/api/pulseser...,В SberData мы создаем централизованное хранили...,· Прототипирование сервисов; \n · Ра...,Опыт разработки Backend (Python) приложений от...,Город Москва,Sber,2022-08-08 14:41:38


In [15]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             1287 non-null   object        
 1   internal_id       1287 non-null   int64         
 2   url               1287 non-null   object        
 3   description       785 non-null    object        
 4   responsibilities  1287 non-null   object        
 5   qualifications    1287 non-null   object        
 6   location          1287 non-null   object        
 7   company           1287 non-null   object        
 8   publish_date      1287 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 90.6+ KB


In [16]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [17]:
snapshot.to_csv(f'../data/sber/{current_date}.csv')
snapshot.to_excel(f'../data/sber/{current_date}.xlsx', engine='xlsxwriter')

In [67]:
requests.get(VACANCY_API_URL + str(jobs_infos[2]), headers=sber_headers, verify=False).json()

{'timestamp': '2022-07-05T06:23:43.116+00:00',
 'status': 400,
 'error': 'Bad Request',
 'message': '',
 'path': '/v1/search/get/1848690'}